In [1]:
import pandas as pd 
import os, warnings
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import display, Markdown

#Local
from config import Config
from data_processing import data_processing
from utils import remove_dirs, check_and_create_dirs
from question_generator import question_generator

#Setting
warnings.filterwarnings("ignore")
# pd.set_option('display.max_colwidth', None)

#CONFIG
load_dotenv()
api_key = os.getenv("PERPLEXITY_API_KEY")

# 1. Initialization

In [2]:
print('INITIALIZING SESSION')
cfg = Config(
    project_name="session_1",# Name of the project
    input_dir="data", # Input directory containing the data files
    api_key= api_key, # API key for Perplexity
    n_questions_per_file=2, #This mean 1 question per file (There are 10 files)
    n_page_summary=3, # Number of pages to summarize
    chunk_size=5000, 
    chunk_overlap=500
)

# remove_dirs(cfg) # This will delete the output directories if they exist
check_and_create_dirs(cfg)


INITIALIZING SESSION
Folder 'session_1' and 'session_1\chunks' has been created.


In [3]:
print("USER'S RAG MODEL EXAMPLE")
display(Markdown("The user having a FAISS-based retrieval-augmented system that chunks PDFs, embeds them with all-mpnet-base-v2, and queries them using the Sonar LLM for question-answering."))

from user_models.qa import create_unified_chain
qa_chain = create_unified_chain()

USER'S RAG MODEL EXAMPLE


The user having a FAISS-based retrieval-augmented system that chunks PDFs, embeds them with all-mpnet-base-v2, and queries them using the Sonar LLM for question-answering.

Creating new vector store for model_1...
Loaded 1182 total documents.
Created 1426 text chunks.
Vector store created and saved to: faiss_index_open
Using standard retriever
QA chain created successfully for model_1


# QUESTION GENERATION

In [4]:
#I QUESTION GENERATION

print('DATA PROCESSING AND CHUNKING ...')
data_processing(api_key, cfg, verbose = False)

print('QUESTION GENERATION ...')
df = question_generator(cfg, verbose = False)
df.to_csv(cfg.question_file, index = False)

print('QUESTION EXAMPLE:')
questions = df['question'].to_list()
display(Markdown(questions[1]))
display(Markdown(questions[2]))

DATA PROCESSING AND CHUNKING ...
Processing file: 2024-amazon-annual-report-10K.pdf 1/10
Processing file: 2024-apple-annual-report-10K.pdf 2/10
Processing file: 2024-cisco-full-annual-report.pdf 3/10
Processing file: 2024-google-annual-report-10K.pdf 4/10
Processing file: 2024-meta-full-annual-report.pdf 5/10
Processing file: 2024-netflix-annual-report-10K.pdf 6/10
Processing file: 2024-nvidia-annual-report-10K.pdf 7/10
Processing file: 2024-oracle-annual-report-10K.pdf 8/10
Processing file: 2024-reddit-annual-report-10K.pdf 9/10
Processing file: 2024-tsla-annual-report-10K.pdf 10/10
QUESTION GENERATION ...
QUESTION EXAMPLE:


How might Amazon's business be affected in 2024 if its policies meant to prevent the sale of counterfeit goods are circumvented?

For Apple Inc. during the fiscal year ending September 28, 2024, what specific challenges does the company face in maintaining its competitive advantage in the global markets for smartphones, personal computers, and tablets?

# GENERATE ANSWER FROM USER MODEL

(Should take about 7s per questions)

In [5]:
print("Number of questions generated:", len(questions))

Number of questions generated: 20


In [6]:
answers = []
for question in df['question']:
    ans = qa_chain.invoke(question)['result'] 
    answers.append(ans)

df['answer'] = answers

# EVALUATION

In [7]:
from evaluation import evaluate_answers, radar_plot, statistics, overall_histogram

print('EVALUATION ...')
final_df = evaluate_answers(api_key, df, verbose = False)

EVALUATION ...


In [8]:
final_df

,question,factual_correctness_score,completeness_score,clarity_score,overall_score
0,Based on the 2024 Amazon report discussing fin...,5,4,5,4.7
1,How might Amazon's business be affected in 202...,5,4,5,4.7
2,For Apple Inc. during the fiscal year ending S...,5,5,5,5.0
3,What is the typical vesting schedule for Restr...,5,4,5,4.7
4,What risks and challenges did Cisco highlight ...,2,2,3,2.2
5,"How did the changes in net income, share repur...",2,3,4,2.7
6,What potential legal and regulatory risks did ...,3,4,5,3.7
7,How does Alphabet Inc. measure and recognize i...,5,5,5,5.0
8,What was the net cash provided by operating ac...,5,5,5,5.0
9,What was the change in the long-term debt of M...,5,5,5,5.0


In [9]:
statistics(final_df)

,count,mean,std,min,median,max
factual_correctness_score,20.00,3.80,1.36,2.00,4.50,5.00
completeness_score,20.00,3.70,1.30,1.00,4.00,5.00
clarity_score,20.00,4.40,0.88,2.00,5.00,5.00
overall_score,20.00,3.89,1.20,1.70,4.45,5.00


In [10]:
radar_plot(final_df)

In [11]:
overall_histogram(final_df)